Course Instructor: Bernd Neumayr, JKU

# UE05: SHACL

Complete the **10 tasks (1 point per task)** in the `4. SHACL` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph) in executable form
- After executing the validation, print out the validation results in tabular form.  


#Preparation

In [ ]:
!pip install -q rdflib 
!pip3 install -q pyshacl

# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.4.1 requires prettytable<1, but you have prettytable 2.5.0 which is incompatible.


#Task 1
Wenn eine Person eine x:knows Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein. 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .  
""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
 sh:targetClass <Person> ;
 sh:property :KnowsShape . 

:KnowsShape a sh:PropertyShape ;
 sh:path :knows ;
 sh:nodeKind sh:IRI .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape :KnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Peter" ] ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,n3616475bbaf34b5a813776a37bdad912b1,sh:NodeKindConstraintComponent,:KnowsShape,Value is not of Node Kind sh:IRI


#Task 2
Die Klassen Man und Woman sind disjunkt. Hinweis: verwenden Sie sh:not. 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .
""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ManShape> a sh:NodeShape ;
 sh:targetClass <Man> ;
 sh:not [
  sh:targetClass <Woman> ;
  sh:class <Woman>
 ] .
 
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Susi ;
            sh:resultMessage "Node :Susi conforms to shape [ sh:class :Woman ; sh:targetClass :Woman ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :Susi ],
        [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Node :John conforms to shape [ sh:class :Woman ; sh:targetClass :Woman ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :John ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Susi,None,:Susi,sh:NotConstraintComponent,:ManShape,Node :Susi conforms to shape [ sh:class :Woman...
1,:John,None,:John,sh:NotConstraintComponent,:ManShape,Node :John conforms to shape [ sh:class :Woman...


#Task 3
Wenn eine Person eine :knows Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.  

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> .
""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
 sh:targetClass <Person> ;
 sh:property :KnowsShape .

:KnowsShape a sh:PropertyShape ;
 sh:path :knows ;
 sh:class <Person> .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :KnowsShape ;
            sh:value :Mary ],
        [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :KnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Mary"

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,n87526feb001d4060badb962c5e05423bb1,sh:ClassConstraintComponent,:KnowsShape,Value does not have class :Person
1,:Peter,:knows,:Mary,sh:ClassConstraintComponent,:KnowsShape,Value does not have class :Person


#Task 4
Jede Person hat genau ein Alter (:age). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse Adult) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie sh:minInclusive und sh:maxInclusive (siehe SHACL Recommendation). 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ABC>   :age    -1 , 27 , 197 .

<John>  rdf:type  <Person> ;
        :age      151 .

<Adult>  rdfs:subClassOf  <Person> .

<Beny>  rdf:type  <Child> ;
        :age      13 .

<Zoe>   rdf:type  <Child> ;
        :age      12 .

<Mary>  rdf:type  <Woman> ;
        :age      17 .

<Bibi>  rdf:type  <Person> ;
        :age      0 .

<Peter>  rdf:type  <Person> .

<Senior>  rdfs:subClassOf  <Adult> .

<Jane>  rdf:type  <Woman> ;
        :age      67 .

<Lili>  rdf:type  <Person> ;
        :age      -1 .

<Senior1>  rdf:type  <Senior> ;
        :age      65 .

<Child>  rdfs:subClassOf  <Person> .

<Bibo>  rdf:type  <Person> ;
        :age      34 , 12 .

<Woman>  rdfs:subClassOf  <Adult> .

<Senior2>  rdf:type  <Senior> ;
        :age      64 .
""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
 sh:targetClass <Person> ;
 sh:property [
  sh:path :age ;
  sh:datatype xsd:integer ;
  sh:maxCount 1 ;
  sh:minCount 1 ;
  sh:minInclusive 0 ;
  sh:maxInclusive 150   
 ] .

:AdultShape a sh:NodeShape ;
 sh:targetClass <Adult> ;
 sh:property [
  sh:path :age ;
  sh:datatype xsd:integer ;
  sh:maxCount 1 ;
  sh:minCount 1 ;
  sh:minInclusive 19 ;
  sh:maxInclusive 150 
 ] .

:ChildShape a sh:NodeShape ;
 sh:targetClass <Child> ;
 sh:property [
  sh:path :age ;
  sh:datatype xsd:integer ;
  sh:maxCount 1 ;
  sh:minCount 1 ;
  sh:minInclusive 0 ;
  sh:maxInclusive 12 
 ] .

:SeniorShape a sh:NodeShape ;
 sh:targetClass <Senior> ;
 sh:property [
  sh:path :age ;
  sh:datatype xsd:integer ;
  sh:maxCount 1 ;
  sh:minCount 1 ;
  sh:minInclusive 65 ;
  sh:maxInclusive 150 
 ] .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Beny ;
            sh:resultMessage "Value is not <= Literal(\"12\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxInclusiveConstraintComponent ;
            sh:sourceShape [ sh:datatype xsd:integer ;
                    sh:maxCount 1 ;
                    sh:maxInclusive 12 ;
                    sh:minCount 1 ;
                    sh:minInclusive 0 ;
                    sh:path :age ] ;
            sh:value 13 ],
        [ a sh:ValidationResult ;
            sh:focusNode :Mary ;
            sh:resultMessage "Value is not >= Literal(\"19\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Senior2,:age,64,sh:MinInclusiveConstraintComponent,n204366d6c0ce423a8ad6a322e6744819b4,"Value is not >= Literal(""65"", datatype=xsd:int..."
1,:Beny,:age,13,sh:MaxInclusiveConstraintComponent,n204366d6c0ce423a8ad6a322e6744819b3,"Value is not <= Literal(""12"", datatype=xsd:int..."
2,:Bibo,:age,None,sh:MaxCountConstraintComponent,n204366d6c0ce423a8ad6a322e6744819b1,More than 1 values on :Bibo->:age
3,:Peter,:age,None,sh:MinCountConstraintComponent,n204366d6c0ce423a8ad6a322e6744819b1,Less than 1 values on :Peter->:age
4,:John,:age,151,sh:MaxInclusiveConstraintComponent,n204366d6c0ce423a8ad6a322e6744819b1,"Value is not <= Literal(""150"", datatype=xsd:in..."
5,:Lili,:age,-1,sh:MinInclusiveConstraintComponent,n204366d6c0ce423a8ad6a322e6744819b1,"Value is not >= Literal(""0"", datatype=xsd:inte..."
6,:Mary,:age,17,sh:MinInclusiveConstraintComponent,n204366d6c0ce423a8ad6a322e6744819b2,"Value is not >= Literal(""19"", datatype=xsd:int..."


#Task 5
Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von :name und rdf:type keine weiteren Eigenschaften haben. 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .

""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
 sh:targetClass <Person> ;
 sh:closed true ;
 sh:property [
  sh:maxCount 1 ;
  sh:path :name ;
  sh:datatype xsd:string ;
  sh:nodeKind sh:Literal
 ] ;
 sh:ignoredProperties ( rdf:type ) .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Jim ;
            sh:resultMessage "Value is not Literal with datatype xsd:string" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:DatatypeConstraintComponent ;
            sh:sourceShape _:n936b8e584c4540f08411aab340eb2a92b1 ;
            sh:value :Jim ],
        [ a sh:ValidationResult ;
            sh:focusNode :Bibi ;
            sh:resultMessage "More than 1 values on :Bibi->:name" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxCountConstraintComponent ;
            sh:sourceShape _:n936b8e584c4540f08411aab340eb2a92b1 ],
        [ a sh:ValidationResult ;
            sh:focusNode :Joh

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:John,:knows,:Mary,sh:ClosedConstraintComponent,:PersonShape,Node :John is closed. It cannot have value: :Mary
1,:Jane,:name,2343,sh:DatatypeConstraintComponent,n936b8e584c4540f08411aab340eb2a92b1,Value is not Literal with datatype xsd:string
2,:Jim,:name,:Jim,sh:DatatypeConstraintComponent,n936b8e584c4540f08411aab340eb2a92b1,Value is not Literal with datatype xsd:string
3,:Jim,:name,:Jim,sh:NodeKindConstraintComponent,n936b8e584c4540f08411aab340eb2a92b1,Value is not of Node Kind sh:Literal
4,:Bibi,:name,None,sh:MaxCountConstraintComponent,n936b8e584c4540f08411aab340eb2a92b1,More than 1 values on :Bibi->:name


#Task 6
Städte sind via Property :inCountry Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind Austria zugeordnet. 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .

""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:AustrianCityShape a sh:NodeShape ;
 sh:targetClass <AustrianCity> ;
 sh:property [
  sh:path :inCountry ;
  sh:in (<Austria>)
 ] . 

:EuropeanCityShape a sh:NodeShape ;
 sh:targetClass <EuropeanCity> ;
 sh:property [
  sh:path :inCountry ;
  sh:class <EuropeanCountry> 
 ] .

:CityShape a sh:NodeShape ;
 sh:targetClass <City> ;
 sh:property [
  sh:path :inCountry ;
  sh:class <Country> 
 ] .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Paris ;
            sh:resultMessage "Value does not have class :Country" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape _:n04185378192b4ef4843a6395ba944e4bb4 ;
            sh:value :France ],
        [ a sh:ValidationResult ;
            sh:focusNode :Paris ;
            sh:resultMessage "Value does not have class :EuropeanCountry" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape [ sh:class :EuropeanCountry ;
      

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Paris,:inCountry,:France,sh:ClassConstraintComponent,n04185378192b4ef4843a6395ba944e4bb3,Value does not have class :EuropeanCountry
1,:Salzburg,:inCountry,:Germany,sh:InConstraintComponent,n04185378192b4ef4843a6395ba944e4bb1,Value :Germany not in list [':Austria']
2,:Paris,:inCountry,:France,sh:ClassConstraintComponent,n04185378192b4ef4843a6395ba944e4bb4,Value does not have class :Country
3,:NewYork,:inCountry,:USA,sh:ClassConstraintComponent,n04185378192b4ef4843a6395ba944e4bb4,Value does not have class :Country


#Task 7
Die Property :worksFor darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.
Hinweis: Lösen Sie die Aufgabe ohne Property Shapes. 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> .

""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:WorksForSubjectsShape
 sh:targetSubjectsOf :worksFor ;  
 sh:class <Person> .

:WorksForTargetsShape
 sh:targetObjectsOf :worksFor ;
 sh:class <Organization> .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Mary ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :WorksForSubjectsShape ;
            sh:value :Mary ],
        [ a sh:ValidationResult ;
            sh:focusNode :LMU ;
            sh:resultMessage "Value does not have class :Organization" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :WorksForTargetsShape ;
            sh:value :LMU ],
        [ a sh:ValidationResult ;
            sh:focusNode :Jim ;
            sh:resultMessage "Value does not have class :Person" ;
            s

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:LMU,None,:LMU,sh:ClassConstraintComponent,:WorksForTargetsShape,Value does not have class :Organization
1,:Mary,None,:Mary,sh:ClassConstraintComponent,:WorksForSubjectsShape,Value does not have class :Person
2,:Jim,None,:Jim,sh:ClassConstraintComponent,:WorksForSubjectsShape,Value does not have class :Person


#Task 8
Peter kennt ausschließlich Männer, die in Österreich leben.

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<John>  rdf:type  <Man> ;
        :livesIn  <Austria> .

<Mary>  rdf:type  <Woman> ;
        :knows    <Jim> ;
        :livesIn  <Austria> .

<Hans>  rdf:type  <Man> .

<Peter>  :knows  <Josef> , <Hans> , <Franz> , <John> , <Mary> .

<Franz>  rdf:type  <Man> ;
        :livesIn  <Germany> .

""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PeterShape a sh:NodeShape ;
 sh:targetNode <Peter> ;
 sh:property [
  sh:path :knows ;
  sh:node :PeterKnowsShape
 ] .

:PeterKnowsShape a sh:NodeShape ;
 sh:class <Man> ;
 sh:property [
  sh:path :livesIn ;
  sh:minCount 1 ;
  sh:in (<Austria>)
 ] .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not conform to Shape :PeterKnowsShape" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape _:n9c63236eb4d447999c8043f2de8aa17db1 ;
            sh:value :Hans ],
        [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not conform to Shape :PeterKnowsShape" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape _:n9c63236eb4d447999c8043f2de8aa17db1 ;
            sh:value :Franz ],
        [ a sh:Val

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,:Hans,sh:NodeConstraintComponent,n9c63236eb4d447999c8043f2de8aa17db1,Value does not conform to Shape :PeterKnowsShape
1,:Peter,:knows,:Josef,sh:NodeConstraintComponent,n9c63236eb4d447999c8043f2de8aa17db1,Value does not conform to Shape :PeterKnowsShape
2,:Peter,:knows,:Franz,sh:NodeConstraintComponent,n9c63236eb4d447999c8043f2de8aa17db1,Value does not conform to Shape :PeterKnowsShape
3,:Peter,:knows,:Mary,sh:NodeConstraintComponent,n9c63236eb4d447999c8043f2de8aa17db1,Value does not conform to Shape :PeterKnowsShape


#Task 9
Das Nettogewicht eines Produkts darf nicht größer sein als das Bruttogewicht.
Verwenden Sie sh:lessThanOrEquals (siehe SHACL Recommendation). 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .

""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:ProductShape a sh:NodeShape ;
 sh:targetClass <Product> ;
 sh:property [
  sh:path :netWeight ;
  sh:lessThanOrEquals :grossWeight
 ] .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Bread ;
            sh:resultMessage "Value of :Bread->:grossWeight < Literal(\"1.2\", datatype=xsd:decimal)" ;
            sh:resultPath :netWeight ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:LessThanOrEqualsConstraintComponent ;
            sh:sourceShape [ sh:lessThanOrEquals :grossWeight ;
                    sh:path :netWeight ] ;
            sh:value 1.2 ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Bread,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,n7da332ba90bb48938c7a8f4ca7fe2ceab1,"Value of :Bread->:grossWeight < Literal(""1.2"",..."


#Task 10
Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). eine 

In [ ]:
dg = Graph()
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Sara>  :givenName  "Sarah" .

<John>  :familyName  "Black" .

<Mary>  :knows  <Pete> .

<Bob>   :familyName  "Builder" ;
        :givenName   "Bob" .

<Pete>  :knows  <Bob> , <Sara> , <John> ;
        :name   "Peter Parker" .

""")

sg = Graph()
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:KnowsShape a sh:NodeShape ;
 sh:targetObjectsOf :knows ;
 sh:or(
  [
   sh:property [
    sh:path :name ;
    sh:minCount 1
   ]
  ]

  [
   sh:property [
    sh:path :givenName ;
    sh:minCount 1
   ], [
    sh:path :familyName ;
    sh:minCount 1
   ]
  ]
 ) .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Sara ;
            sh:resultMessage "Node :Sara does not conform to one or more shapes in [ sh:property [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :name ] ] , [ sh:property [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :familyName ], [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :givenName ] ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:OrConstraintComponent ;
            sh:sourceShape :KnowsShape ;
            sh:value :Sara ],
        [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Node :John does not conform to one or more shapes in [ sh:property [ sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:pat

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Sara,None,:Sara,sh:OrConstraintComponent,:KnowsShape,Node :Sara does not conform to one or more sha...
1,:John,None,:John,sh:OrConstraintComponent,:KnowsShape,Node :John does not conform to one or more sha...
